# Visualization

---

## Learning Objectives 


- How to use xarray's builtin, matplotlib-backed plotting interface to visualize datasets.
- How to use `hvplot` to produce interactive plots 

## Prerequisites


| Concepts | Importance | Notes |
| --- | --- | --- |
| [Understanding of xarray core data structures](./01-xarray-fundamentals.ipynb) | Necessary | |
| [Familiarity with xarray indexing and subsetting](./02-indexing-and-subsetting.ipynb) | Necessary | |
| [Basic familiarity with Matplotlib](https://numpy.org/doc/stable/reference/arrays.indexing.html) | Helpful | |


- **Time to learn**: *medium*



---

## Imports


In [ ]:
import xarray as xr

Let's open the same dataset as before

In [ ]:
ds = xr.open_dataset(
    "./data/tas_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc", engine="netcdf4"
)
ds

## Basic plotting with via `.plot()`

Xarray provides a `.plot()` method on `DataArray` and `Dataset`. This method is a wrapper around Matplotlib's `matplotlib.pyplot.plot()`. xaarray will automatically guess the type of plot based on the dimensionality of the data. By default `.plot()` creates:

- a **line** plot for `1-D arrays` using `matplotlib.pyplot.plot()`
- a **pcolormesh** plot for 2-D arrays using `matplotlib.pyplot.pcolormesh()`
- a **histogram** for everything else (more than 2 dimensions) using `matplotlib.pyplot.hist()`
 

### 1D line plots

Let's select one spatial location and plot a time sesries of the near surface temperature

In [ ]:
ds.tas.sel(lon=100, lat=10, method='nearest').plot(marker="o", size=6);

<div class="admonition alert alert-info">
    We are selecting a single point, so `.sel()` requires either an exact location that exists in the data, or to specify method argument to tell it how to choose a location from the data. 
</div>


Lets say we want to compare plots of temperature at three different latitudes. We can use the `hue` keyword argument to do this.

In [ ]:
ds.tas.sel(lat=[-40, 0, 40], time="2013-03", method="nearest")

In [ ]:
ds.tas.sel(lat=[-40, 0, 40], time="2013-03", method="nearest").plot(
    x="lon", hue="lat", figsize=(8, 6)
);

### 2D plots

Operator chaining means it is possible to have multiple selection operators and add `.plot()` to the end to visualise the result

In [ ]:
ds.tas.isel(time=-10).sel(lon=slice(20, 160), lat=slice(-80, 25)).plot(robust=True, figsize=(8, 6));

The x- and y-axes are labeled with full names — "Latitude", "Longitude" — along with units. The colorbar has a nice label, again with units. And the title tells us the timestamp of the data presented.

In [ ]:
# define keyword arguments that are passed to matptolib.pyplot.colorbar
colorbar_kwargs = {
    "orientation": "horizontal",
    "label": "my clustom label",
    "pad": 0.2,
}

ds.tas.isel(lon=1).plot(
    x="time",  # coordinate to plot on the x-axis of the plot
    robust=True,  # set colorbar limits to 2nd and 98th percentile of data
    cbar_kwargs=colorbar_kwargs,
);


### Faceting

Faceting is an effective way of visualizing variations of 3D data where 2D slices are visualized in a panel (subplot) and the third dimensions is varied between panels (subplots).

In [ ]:
ds.tas.sel(time=slice("2010", "2011")).plot(col="time", col_wrap=6, robust=True);

See the [xarray documentation](https://xarray.pydata.org/en/stable/user-guide/plotting.html) for more on "faceted" plots or subplots.

### Histograms

In [ ]:
ds.tas.plot();

### Bonus Plot 

Let's look at the air temperature data but at for **all pressure levels**. We are going to select out the first time index and the longitude corresponding to the Himalayas and plot a vertical profile of the atmosphere from pole to pole:

In [ ]:
ds_air_all_pressure_levels = xr.open_dataset(
    "data/ta_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc", engine="netcdf4"
)
ds_air_all_pressure_levels

In [ ]:
data = ds_air_all_pressure_levels.ta.isel(time=-1).sel(lon=86.93, method='nearest')
fig = data.plot(size=6, yincrease=False)
fig.axes.set_title(
    f'Vertical profile of Temperature from pole to pole \nat longitude = {data.lon.data} and time = {data.time.data}',
    size=15,
);

<div class="admonition alert alert-info">
     <ul>
         <li>The yincrease=False option was used for the plot to invert the y-axis as pressure decreases with height</li>
         <li>We can more complicated figures and/or make customizations to our plots by saving the returned object from .plot and accessing the .axes attribute of the returned object</li>
    </ul>
</div>

---

## Interactive visualization using `hvplot`

Let's switch gears and look at how we can produce interactive plots via [holoviews](https://holoviews.org/). The holoviews plotting ecosystem provides the [hvplot](https://hvplot.holoviz.org/) package to allow easy visualization of xarray (and other) objects. These plots build on [Bokeh](https://docs.bokeh.org/en/latest/index.html) -- a Python library for creating interactive visualziatons for web browsers.


To enable the `.hvplot` interface on xarray object, let's import the `hvplot.xarray` module:

In [ ]:
import hvplot.xarray

To use `hvplot` instead of `matplotlib`, we use the `.hvplot()` method:

In [ ]:
ds.tas.hvplot()

As you can see, calling `.hvplot()` behaves the same as `.plot()` i.e. it uses the same heuristics as `.plot()`. In this case, it produces a histogram for data with more than 3 dimensions. To plot a `pcolormesh`, tet's reduce the dimensionality of our data to 2D and call `.hvplot()` again:

In [ ]:
ds.tas.isel(time=1).hvplot(cmap="fire")

In [ ]:
ds.tas.isel(time=-1, lon=100).hvplot()

In [ ]:
ds.tas.sel(lat=28.5, lon=83.9, method='nearest').hvplot()

So far we have had to subset our data in order to produce plots. `hvplot` provides convenient functionality for producing plots on-demand by via interactive widgets. Let's create a series of 2D for each time slice, We will use the `groupby` parameter to let hvplot know that we want to create a widget (a slider) for the time dimension:

In [ ]:
ds.tas.hvplot(groupby="time", clim=(ds.tas.min(), ds.tas.max()), cmap='turbo')

Let's add more customizations to our time widget:

In [ ]:
ds.tas.hvplot(
    groupby="time",
    clim=(ds.tas.min(), ds.tas.max()),
    cmap="turbo",
    widget_type="scrubber",
    widget_location="bottom",
)

---

In [ ]:
%load_ext watermark
%watermark --time --python --updated --iversion

## Summary 

- Xarray has plotting functionality that is a thin wrapper around the Matplotlib library
- Xarray uses syntax and function names from Matplotlib whenever possible
- Hvplot provides a neat interface to xarray for creating interactive plots

## Resources and References

- [Hvplot Documentation](https://hvplot.holoviz.org/index.html)
- [Xarray Documentation - Plotting](https://xarray.pydata.org/en/stable/user-guide/plotting.html)
- [Matplolib Documentation](https://matplotlib.org/stable/contents.html)

<div class="admonition alert alert-info">
    <p class="title" style="font-weight:bold">Geocat-examples Gallery</p>
    For geo-science specific visualization examples, please see the geocat-examples gallery which resides <a href="https://geocat-examples.readthedocs.io/">here</a>.
</div>


<div class="admonition alert alert-success">
    <p class="title" style="font-weight:bold">Previous: <a href="./01-xarray-fundamentals.ipynb">Xarray Fundamentals</a></p>
         <p class="title" style="font-weight:bold">Next: <a href="./04-computation.ipynb">Computation</a></p>
</div>